In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install transformers

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
torch.set_default_device("cuda")

model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2", torch_dtype="auto", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

In [4]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_name = "microsoft/phi-2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

ValueError: Unrecognized configuration class <class 'transformers.models.phi.configuration_phi.PhiConfig'> for this kind of AutoModel: AutoModelForSeq2SeqLM.
Model type should be one of BartConfig, BigBirdPegasusConfig, BlenderbotConfig, BlenderbotSmallConfig, EncoderDecoderConfig, FSMTConfig, GPTSanJapaneseConfig, LEDConfig, LongT5Config, M2M100Config, MarianConfig, MBartConfig, MT5Config, MvpConfig, NllbMoeConfig, PegasusConfig, PegasusXConfig, PLBartConfig, ProphetNetConfig, SeamlessM4TConfig, SeamlessM4Tv2Config, SwitchTransformersConfig, T5Config, UMT5Config, XLMProphetNetConfig.

In [6]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

def summarize_text(text, model_name="microsoft/phi-2"):
  """
  This function takes text and a model name as input and attempts to generate a summary using zero-shot prompting.

  Args:
      text: Generate Summary.
      model_name: The name of the language model to use. (Default: "microsoft/phi-2")

  Returns:
      A string containing the generated summary, or an empty string if the summarization fails.
  """

  try:
    # Load the model and tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

    # Important note:
    # If you're using Transformers version below 4.37.0, you need to pass an additional argument
    # to prevent potential side-effects due to remote code execution:
    # model = AutoModelForSeq2SeqLM.from_pretrained(model_name, trust_remote_code=True)

    # Craft a clear and informative prompt
    prompt = f"""
    **Please provide a concise summary of the following text, capturing the main points and keeping the length around 3 sentences.**

    {text}
    """

    # Tokenize and generate text
    input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"]
    output = model.generate(input_ids)
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

    return generated_text.strip()  # Remove potential leading/trailing whitespaces

  except Exception as e:
    print(f"Error during summarization: {e}")
    return ""

# Example usage
text = '''(CNN) -- My mom was always sports mad -- my dad not so much. But both my parents understood the importance of significant events, so when opportunity knocked, my parents opened up all doors for me. I grew up knowing full well that my country wasn't allowed to participate in international sports following the sporting boycott of 1977 which had been placed on South Africa in a protest against apartheid -- a system of racial segregation enforced by the ruling white minority. So when the South African rugby team toured New Zealand in 1981, my parents made sure that I was awoken at 04:30 to watch all three Test matches. I got to snuggle between my parents -- and a first -- they put the television in their bedroom. It was the ""flour bomb"" Test -- where a light aircraft flew over Auckland's Eden Park before and during the match, dropping flour bombs, smoke bombs and anti-apartheid leaflets -- where I began to realize that other countries didn't like us. I didn't understand why at the time, and it hurt a little. Later that day, I'll never forget my parents telling me that it could be a long time before I witnessed something like this again. I was six at the time. They were right; I would next see these two teams play as a 17-year-old. First trip . My first trip to Cape Town's Newlands Cricket Ground was to watch Western Province play a domestic match. It must have been my mom's first trip too, because I distinctly remember entering the ground and ending up sitting on the grass embankment, which was reserved for colored people. Confused, we sat down. A kind couple offered us some fruit, so we stayed. I was blissfully unaware of how significant this possibly was. We returned a couple of years later to watch my first ""Test"" match: South Africa vs. the West Indies. This time we were seated in the correct place, in the Oaks Enclosure, which ironically these days is a grass embankment. I remember the hype and the excitement about this special day. I had seen the Windies in their warmup game against Western Province and I knew that they were something special. They had vicious fast bowlers and flamboyant batsmen. We got there early, along with thousands of others, just to watch the warmups. The excitement was tangible. I don't remember much about the games -- I remember more about the players. They were heroes to us. Perhaps there was some curiosity too, something about the islanders from far, far away on our shores. We used to follow them around the boundaries of the ground, hunting their prized autographs. And their accents -- just too cool. Didn't see color . I know I didn't see the color. I don't think most kids did -- they were just cricketers. At the end of play we would run on the field to try to touch them and grab one more autograph. Somewhere there is footage of me congratulating Sylvester Clarke by patting his back. I remember telling all my friends that I had touched the great bowler. While I had forgotten about it, listening to Franklyn Stephenson speak reminded me of the time we were trying to get him to drink from some kid's Coke. He obliged and we all simply went crazy. I realize the significance of that event now. Imagine a black guy drinking from the same can? Perhaps our parents would have flipped out, but this wasn't about color to us. Other memories include how obliging and friendly they were. Happy to give autographs and happy to interact positively with the huge crowds. Branded a rebel . It was with incredible interest and fascination that I watched ""Branded a rebel"" -- a CNN World Sport documentary which first screened last year. It gave me greater insight into the players, the tour, West Indies cricket and indeed my home country. And while ""Branded a rebel"" is an appropriate title for the program, it could have just as easily been titled ""Branded Heroes."" That's what they were to us white South Africans, and I think deep down the Windies players felt that too. It's all about what side of the line you were sitting, I guess. On their return home, the West Indian players were shunned by society and some were forced to move abroad to make a living. The poverty and other such ills which befell some of their players really tugged at all of my emotions. I started thinking what we as South Africans could have done for them. Could we have done more? Did we look after them well enough after the fact? It's a difficult one. These men didn't deserve it, no matter what choices they made. Society can be cruel. Strangled by hatred . It eats me up to think what those players endured upon their return. They were strangled by such hatred in their home countries, and yet I wish they somehow knew what they had done for young cricket fans like me. I got to see ""real"" international cricket played at the ""highest"" level. I got to meet new cricketing greats. I got to bowl like Clarke, Collis King and Colin Croft. I got to emulate David Murray's great wicketkeeping skills on Clifton beach, field like Alvin Kallicharan and admire the patience and masterful batting of the great Lawrence Rowe. And something else -- they taught us not to take ourselves too seriously by showing how much one can enjoy themselves on the sports field. Their smiles, I'll never forget their smiles. Each of their names is entrenched in my memory, and of all of those who saw the matches. They became part of my upbringing and that of thousands of South African fans. What I didn't understand at the time, was what it took for those men to actually land on our shores (money aside). They must have been scared -- really scared. I know I would have been. In the interview with Clive Lloyd, longtime captain of the official West Indies team that dominated world cricket for two decades, he said no amount of money would have made him support a racist regime. I would go as far as to say no amount of money would have put my fears at ease to travel to South Africa. Courage . It's not for me to judge, but those men were courageous. In their cricket and in their collective decisions, and I respect that. A few years later, I did wonder how the black West Indian players got to travel using ""white"" transport and stay in ""white"" areas. I remember asking my mom how that worked. She didn't know, but you could see that my political curiosity was growing. I was disappointed that more players didn't take up the opportunity of being interviewed for the documentary. I was intrigued by Stephenson, King and Murray's thoughts, motives and reflective answers. There seemed no real regret, which was fascinating and pleasing for some reason. As for those who declined an interview, their wounds obviously cut deep. Stephenson was just how I remember him. Boisterous, bubbly and always looking on the bright side of life. We got to know him well here, as he returned to play for the Orange Free State. The fact that he still has all his memorabilia means that the tour meant something to him, and that he feels part of change and change of attitudes in South Africa. I don't know if they changed us as a society but I'd like to thing so, even if it was an indescribably fractured and dysfunctional society. And by this I mean that white South Africa packed out every cricket ground to watch black West Indies, went home and made sure that ""The Cosby Show"" was the most watched program on television -- and yet black people were pretty much denied and excluded from everything. Heart bleeds . ""Branded a rebel"" showed balanced reporting. I like the fact that you saw the tour from a South African perspective as well. Great players like Graeme Pollock, Barry Richards and Clive Rice also effectively served life bans for being, well, South African, and the program understood that. I know that, fundamentally, the tour was morally corrupt, but for an innocent nine-year-old devoid of political allegiances and beliefs, it was simply a chance to see the world's best cricketers entertaining us on the world's most beautiful stage. Ultimately, I'd also like to believe that South African cricketers and administrators were trying to show the government the way forward, and that unity through sport was possible.''' # Replace with your actual text
summary = summarize_text(text)

if summary:
  print("Summary:", summary)
else:
  print("Summarization failed.")


Error during summarization: Unrecognized configuration class <class 'transformers.models.phi.configuration_phi.PhiConfig'> for this kind of AutoModel: AutoModelForSeq2SeqLM.
Model type should be one of BartConfig, BigBirdPegasusConfig, BlenderbotConfig, BlenderbotSmallConfig, EncoderDecoderConfig, FSMTConfig, GPTSanJapaneseConfig, LEDConfig, LongT5Config, M2M100Config, MarianConfig, MBartConfig, MT5Config, MvpConfig, NllbMoeConfig, PegasusConfig, PegasusXConfig, PLBartConfig, ProphetNetConfig, SeamlessM4TConfig, SeamlessM4Tv2Config, SwitchTransformersConfig, T5Config, UMT5Config, XLMProphetNetConfig.
Summarization failed.


# **Try 2**

In [7]:
!pip install transformers sentencepiece accelerate bitsandbytes einops -qq -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.9/330.9 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 89.9 MB/s eta 0:00:00


In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [9]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code = True)

In [10]:
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/phi-2",
    torch_dtype = "auto",
    device_map = "auto",
    trust_remote_code = True
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
prompt = """
Instruct: Generate summary from this following article within 100 to 120 wrods.
Output:

Article: My mom was always sports mad -- my dad not so much. But both my parents understood the importance of significant events, so when opportunity knocked, my parents opened up all doors for me. I grew up knowing full well that my country wasn't allowed to participate in international sports following the sporting boycott of 1977 which had been placed on South Africa in a protest against apartheid -- a system of racial segregation enforced by the ruling white minority. So when the South African rugby team toured New Zealand in 1981, my parents made sure that I was awoken at 04:30 to watch all three Test matches. I got to snuggle between my parents -- and a first -- they put the television in their bedroom. It was the ""flour bomb"" Test -- where a light aircraft flew over Auckland's Eden Park before and during the match, dropping flour bombs, smoke bombs and anti-apartheid leaflets -- where I began to realize that other countries didn't like us. I didn't understand why at the time, and it hurt a little. Later that day, I'll never forget my parents telling me that it could be a long time before I witnessed something like this again. I was six at the time. They were right; I would next see these two teams play as a 17-year-old. First trip . My first trip to Cape Town's Newlands Cricket Ground was to watch Western Province play a domestic match. It must have been my mom's first trip too, because I distinctly remember entering the ground and ending up sitting on the grass embankment, which was reserved for colored people. Confused, we sat down. A kind couple offered us some fruit, so we stayed. I was blissfully unaware of how significant this possibly was. We returned a couple of years later to watch my first ""Test"" match: South Africa vs. the West Indies. This time we were seated in the correct place, in the Oaks Enclosure, which ironically these days is a grass embankment. I remember the hype and the excitement about this special day. I had seen the Windies in their warmup game against Western Province and I knew that they were something special. They had vicious fast bowlers and flamboyant batsmen. We got there early, along with thousands of others, just to watch the warmups. The excitement was tangible. I don't remember much about the games -- I remember more about the players. They were heroes to us. Perhaps there was some curiosity too, something about the islanders from far, far away on our shores. We used to follow them around the boundaries of the ground, hunting their prized autographs. And their accents -- just too cool. Didn't see color . I know I didn't see the color. I don't think most kids did -- they were just cricketers. At the end of play we would run on the field to try to touch them and grab one more autograph. Somewhere there is footage of me congratulating Sylvester Clarke by patting his back. I remember telling all my friends that I had touched the great bowler. While I had forgotten about it, listening to Franklyn Stephenson speak reminded me of the time we were trying to get him to drink from some kid's Coke. He obliged and we all simply went crazy. I realize the significance of that event now. Imagine a black guy drinking from the same can? Perhaps our parents would have flipped out, but this wasn't about color to us. Other memories include how obliging and friendly they were. Happy to give autographs and happy to interact positively with the huge crowds. Branded a rebel . It was with incredible interest and fascination that I watched ""Branded a rebel"" -- a CNN World Sport documentary which first screened last year. It gave me greater insight into the players, the tour, West Indies cricket and indeed my home country. And while ""Branded a rebel"" is an appropriate title for the program, it could have just as easily been titled ""Branded Heroes."" That's what they were to us white South Africans, and I think deep down the Windies players felt that too. It's all about what side of the line you were sitting, I guess. On their return home, the West Indian players were shunned by society and some were forced to move abroad to make a living. The poverty and other such ills which befell some of their players really tugged at all of my emotions. I started thinking what we as South Africans could have done for them. Could we have done more? Did we look after them well enough after the fact? It's a difficult one. These men didn't deserve it, no matter what choices they made. Society can be cruel. Strangled by hatred . It eats me up to think what those players endured upon their return. They were strangled by such hatred in their home countries, and yet I wish they somehow knew what they had done for young cricket fans like me. I got to see ""real"" international cricket played at the ""highest"" level. I got to meet new cricketing greats. I got to bowl like Clarke, Collis King and Colin Croft. I got to emulate David Murray's great wicketkeeping skills on Clifton beach, field like Alvin Kallicharan and admire the patience and masterful batting of the great Lawrence Rowe. And something else -- they taught us not to take ourselves too seriously by showing how much one can enjoy themselves on the sports field. Their smiles, I'll never forget their smiles. Each of their names is entrenched in my memory, and of all of those who saw the matches. They became part of my upbringing and that of thousands of South African fans. What I didn't understand at the time, was what it took for those men to actually land on our shores (money aside). They must have been scared -- really scared. I know I would have been. In the interview with Clive Lloyd, longtime captain of the official West Indies team that dominated world cricket for two decades, he said no amount of money would have made him support a racist regime. I would go as far as to say no amount of money would have put my fears at ease to travel to South Africa. Courage . It's not for me to judge, but those men were courageous. In their cricket and in their collective decisions, and I respect that. A few years later, I did wonder how the black West Indian players got to travel using ""white"" transport and stay in ""white"" areas. I remember asking my mom how that worked. She didn't know, but you could see that my political curiosity was growing. I was disappointed that more players didn't take up the opportunity of being interviewed for the documentary. I was intrigued by Stephenson, King and Murray's thoughts, motives and reflective answers. There seemed no real regret, which was fascinating and pleasing for some reason. As for those who declined an interview, their wounds obviously cut deep. Stephenson was just how I remember him. Boisterous, bubbly and always looking on the bright side of life. We got to know him well here, as he returned to play for the Orange Free State. The fact that he still has all his memorabilia means that the tour meant something to him, and that he feels part of change and change of attitudes in South Africa. I don't know if they changed us as a society but I'd like to thing so, even if it was an indescribably fractured and dysfunctional society. And by this I mean that white South Africa packed out every cricket ground to watch black West Indies, went home and made sure that ""The Cosby Show"" was the most watched program on television -- and yet black people were pretty much denied and excluded from everything. Heart bleeds . ""Branded a rebel"" showed balanced reporting. I like the fact that you saw the tour from a South African perspective as well. Great players like Graeme Pollock, Barry Richards and Clive Rice also effectively served life bans for being, well, South African, and the program understood that. I know that, fundamentally, the tour was morally corrupt, but for an innocent nine-year-old devoid of political allegiances and beliefs, it was simply a chance to see the world's best cricketers entertaining us on the world's most beautiful stage. Ultimately, I'd also like to believe that South African cricketers and administrators were trying to show the government the way forward, and that unity through sport was possible.
"""

In [12]:
prompt = """
Generate a short summary from this following text.
Text: My mom was always sports mad -- my dad not so much. But both my parents understood the importance of significant events, so when opportunity knocked, my parents opened up all doors for me. I grew up knowing full well that my country wasn't allowed to participate in international sports following the sporting boycott of 1977 which had been placed on South Africa in a protest against apartheid -- a system of racial segregation enforced by the ruling white minority. So when the South African rugby team toured New Zealand in 1981, my parents made sure that I was awoken at 04:30 to watch all three Test matches. I got to snuggle between my parents -- and a first -- they put the television in their bedroom. It was the ""flour bomb"" Test -- where a light aircraft flew over Auckland's Eden Park before and during the match, dropping flour bombs, smoke bombs and anti-apartheid leaflets -- where I began to realize that other countries didn't like us. I didn't understand why at the time, and it hurt a little. Later that day, I'll never forget my parents telling me that it could be a long time before I witnessed something like this again. I was six at the time. They were right; I would next see these two teams play as a 17-year-old. First trip . My first trip to Cape Town's Newlands Cricket Ground was to watch Western Province play a domestic match. It must have been my mom's first trip too, because I distinctly remember entering the ground and ending up sitting on the grass embankment, which was reserved for colored people. Confused, we sat down. A kind couple offered us some fruit, so we stayed. I was blissfully unaware of how significant this possibly was. We returned a couple of years later to watch my first ""Test"" match: South Africa vs. the West Indies. This time we were seated in the correct place, in the Oaks Enclosure, which ironically these days is a grass embankment. I remember the hype and the excitement about this special day. I had seen the Windies in their warmup game against Western Province and I knew that they were something special. They had vicious fast bowlers and flamboyant batsmen. We got there early, along with thousands of others, just to watch the warmups. The excitement was tangible. I don't remember much about the games -- I remember more about the players. They were heroes to us. Perhaps there was some curiosity too, something about the islanders from far, far away on our shores. We used to follow them around the boundaries of the ground, hunting their prized autographs. And their accents -- just too cool. Didn't see color . I know I didn't see the color. I don't think most kids did -- they were just cricketers. At the end of play we would run on the field to try to touch them and grab one more autograph. Somewhere there is footage of me congratulating Sylvester Clarke by patting his back. I remember telling all my friends that I had touched the great bowler. While I had forgotten about it, listening to Franklyn Stephenson speak reminded me of the time we were trying to get him to drink from some kid's Coke. He obliged and we all simply went crazy. I realize the significance of that event now. Imagine a black guy drinking from the same can? Perhaps our parents would have flipped out, but this wasn't about color to us. Other memories include how obliging and friendly they were. Happy to give autographs and happy to interact positively with the huge crowds. Branded a rebel . It was with incredible interest and fascination that I watched ""Branded a rebel"" -- a CNN World Sport documentary which first screened last year. It gave me greater insight into the players, the tour, West Indies cricket and indeed my home country. And while ""Branded a rebel"" is an appropriate title for the program, it could have just as easily been titled ""Branded Heroes."" That's what they were to us white South Africans, and I think deep down the Windies players felt that too. It's all about what side of the line you were sitting, I guess. On their return home, the West Indian players were shunned by society and some were forced to move abroad to make a living. The poverty and other such ills which befell some of their players really tugged at all of my emotions. I started thinking what we as South Africans could have done for them. Could we have done more? Did we look after them well enough after the fact? It's a difficult one. These men didn't deserve it, no matter what choices they made. Society can be cruel. Strangled by hatred . It eats me up to think what those players endured upon their return. They were strangled by such hatred in their home countries, and yet I wish they somehow knew what they had done for young cricket fans like me. I got to see ""real"" international cricket played at the ""highest"" level. I got to meet new cricketing greats. I got to bowl like Clarke, Collis King and Colin Croft. I got to emulate David Murray's great wicketkeeping skills on Clifton beach, field like Alvin Kallicharan and admire the patience and masterful batting of the great Lawrence Rowe. And something else -- they taught us not to take ourselves too seriously by showing how much one can enjoy themselves on the sports field. Their smiles, I'll never forget their smiles. Each of their names is entrenched in my memory, and of all of those who saw the matches. They became part of my upbringing and that of thousands of South African fans. What I didn't understand at the time, was what it took for those men to actually land on our shores (money aside). They must have been scared -- really scared. I know I would have been. In the interview with Clive Lloyd, longtime captain of the official West Indies team that dominated world cricket for two decades, he said no amount of money would have made him support a racist regime. I would go as far as to say no amount of money would have put my fears at ease to travel to South Africa. Courage . It's not for me to judge, but those men were courageous. In their cricket and in their collective decisions, and I respect that. A few years later, I did wonder how the black West Indian players got to travel using ""white"" transport and stay in ""white"" areas. I remember asking my mom how that worked. She didn't know, but you could see that my political curiosity was growing. I was disappointed that more players didn't take up the opportunity of being interviewed for the documentary. I was intrigued by Stephenson, King and Murray's thoughts, motives and reflective answers. There seemed no real regret, which was fascinating and pleasing for some reason. As for those who declined an interview, their wounds obviously cut deep. Stephenson was just how I remember him. Boisterous, bubbly and always looking on the bright side of life. We got to know him well here, as he returned to play for the Orange Free State. The fact that he still has all his memorabilia means that the tour meant something to him, and that he feels part of change and change of attitudes in South Africa. I don't know if they changed us as a society but I'd like to thing so, even if it was an indescribably fractured and dysfunctional society. And by this I mean that white South Africa packed out every cricket ground to watch black West Indies, went home and made sure that ""The Cosby Show"" was the most watched program on television -- and yet black people were pretty much denied and excluded from everything. Heart bleeds . ""Branded a rebel"" showed balanced reporting. I like the fact that you saw the tour from a South African perspective as well. Great players like Graeme Pollock, Barry Richards and Clive Rice also effectively served life bans for being, well, South African, and the program understood that. I know that, fundamentally, the tour was morally corrupt, but for an innocent nine-year-old devoid of political allegiances and beliefs, it was simply a chance to see the world's best cricketers entertaining us on the world's most beautiful stage. Ultimately, I'd also like to believe that South African cricketers and administrators were trying to show the government the way forward, and that unity through sport was possible.

Summary:
"""

In [13]:
with torch.no_grad():
  token_ids = tokenizer.encode(prompt, add_special_tokens=False ,return_tensors="pt")
  output_ids = model.generate(
      token_ids.to(model.device),
      max_new_tokens=512,
      do_sample=True,
      temperature = 0.1
  )

output = tokenizer.decode(output_ids[0][token_ids.size(1) :])
print(output)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (2048). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


The author, a young South African, recounts their experience of watching the 1981 South African rugby tour of New Zealand. They describe the excitement and anticipation leading up to the matches, as well as the disappointment and confusion they felt when they realized the tour was taking place during a time of racial segregation in South Africa. The author also reflects on the impact the tour had on their perception of race and the players involved. They express admiration for the courage of the black West Indian players who traveled to South Africa, despite the risks involved. The author also discusses their curiosity about how the players were able to travel and stay in South Africa during a time of racial discrimination. They mention their disappointment that some players declined to be interviewed for a documentary about the tour, and their hope that the tour helped to bring about change in South Africa. Overall, the author's experience of the tour highlights the complexities of ra

In [ ]:
prompt = """
Generate a short summary from this following text.
Text: My mom was always sports mad -- my dad not so much. But both my parents understood the importance of significant events, so when opportunity knocked, my parents opened up all doors for me. I grew up knowing full well that my country wasn't allowed to participate in international sports following the sporting boycott of 1977 which had been placed on South Africa in a protest against apartheid -- a system of racial segregation enforced by the ruling white minority. So when the South African rugby team toured New Zealand in 1981, my parents made sure that I was awoken at 04:30 to watch all three Test matches. I got to snuggle between my parents -- and a first -- they put the television in their bedroom. It was the ""flour bomb"" Test -- where a light aircraft flew over Auckland's Eden Park before and during the match, dropping flour bombs, smoke bombs and anti-apartheid leaflets -- where I began to realize that other countries didn't like us. I didn't understand why at the time, and it hurt a little. Later that day, I'll never forget my parents telling me that it could be a long time before I witnessed something like this again. I was six at the time. They were right; I would next see these two teams play as a 17-year-old. First trip . My first trip to Cape Town's Newlands Cricket Ground was to watch Western Province play a domestic match. It must have been my mom's first trip too, because I distinctly remember entering the ground and ending up sitting on the grass embankment, which was reserved for colored people. Confused, we sat down. A kind couple offered us some fruit, so we stayed. I was blissfully unaware of how significant this possibly was. We returned a couple of years later to watch my first ""Test"" match: South Africa vs. the West Indies. This time we were seated in the correct place, in the Oaks Enclosure, which ironically these days is a grass embankment. I remember the hype and the excitement about this special day. I had seen the Windies in their warmup game against Western Province and I knew that they were something special. They had vicious fast bowlers and flamboyant batsmen. We got there early, along with thousands of others, just to watch the warmups. The excitement was tangible. I don't remember much about the games -- I remember more about the players. They were heroes to us. Perhaps there was some curiosity too, something about the islanders from far, far away on our shores. We used to follow them around the boundaries of the ground, hunting their prized autographs. And their accents -- just too cool. Didn't see color . I know I didn't see the color. I don't think most kids did -- they were just cricketers. At the end of play we would run on the field to try to touch them and grab one more autograph. Somewhere there is footage of me congratulating Sylvester Clarke by patting his back. I remember telling all my friends that I had touched the great bowler. While I had forgotten about it, listening to Franklyn Stephenson speak reminded me of the time we were trying to get him to drink from some kid's Coke. He obliged and we all simply went crazy. I realize the significance of that event now. Imagine a black guy drinking from the same can? Perhaps our parents would have flipped out, but this wasn't about color to us. Other memories include how obliging and friendly they were. Happy to give autographs and happy to interact positively with the huge crowds. Branded a rebel . It was with incredible interest and fascination that I watched ""Branded a rebel"" -- a CNN World Sport documentary which first screened last year. It gave me greater insight into the players, the tour, West Indies cricket and indeed my home country. And while ""Branded a rebel"" is an appropriate title for the program, it could have just as easily been titled ""Branded Heroes."" That's what they were to us white South Africans, and I think deep down the Windies players felt that too. It's all about what side of the line you were sitting, I guess. On their return home, the West Indian players were shunned by society and some were forced to move abroad to make a living. The poverty and other such ills which befell some of their players really tugged at all of my emotions. I started thinking what we as South Africans could have done for them. Could we have done more? Did we look after them well enough after the fact? It's a difficult one. These men didn't deserve it, no matter what choices they made. Society can be cruel. Strangled by hatred . It eats me up to think what those players endured upon their return. They were strangled by such hatred in their home countries, and yet I wish they somehow knew what they had done for young cricket fans like me. I got to see ""real"" international cricket played at the ""highest"" level. I got to meet new cricketing greats. I got to bowl like Clarke, Collis King and Colin Croft. I got to emulate David Murray's great wicketkeeping skills on Clifton beach, field like Alvin Kallicharan and admire the patience and masterful batting of the great Lawrence Rowe. And something else -- they taught us not to take ourselves too seriously by showing how much one can enjoy themselves on the sports field. Their smiles, I'll never forget their smiles. Each of their names is entrenched in my memory, and of all of those who saw the matches. They became part of my upbringing and that of thousands of South African fans. What I didn't understand at the time, was what it took for those men to actually land on our shores (money aside). They must have been scared -- really scared. I know I would have been. In the interview with Clive Lloyd, longtime captain of the official West Indies team that dominated world cricket for two decades, he said no amount of money would have made him support a racist regime. I would go as far as to say no amount of money would have put my fears at ease to travel to South Africa. Courage . It's not for me to judge, but those men were courageous. In their cricket and in their collective decisions, and I respect that. A few years later, I did wonder how the black West Indian players got to travel using ""white"" transport and stay in ""white"" areas. I remember asking my mom how that worked. She didn't know, but you could see that my political curiosity was growing. I was disappointed that more players didn't take up the opportunity of being interviewed for the documentary. I was intrigued by Stephenson, King and Murray's thoughts, motives and reflective answers. There seemed no real regret, which was fascinating and pleasing for some reason. As for those who declined an interview, their wounds obviously cut deep. Stephenson was just how I remember him. Boisterous, bubbly and always looking on the bright side of life. We got to know him well here, as he returned to play for the Orange Free State. The fact that he still has all his memorabilia means that the tour meant something to him, and that he feels part of change and change of attitudes in South Africa. I don't know if they changed us as a society but I'd like to thing so, even if it was an indescribably fractured and dysfunctional society. And by this I mean that white South Africa packed out every cricket ground to watch black West Indies, went home and made sure that ""The Cosby Show"" was the most watched program on television -- and yet black people were pretty much denied and excluded from everything. Heart bleeds . ""Branded a rebel"" showed balanced reporting. I like the fact that you saw the tour from a South African perspective as well. Great players like Graeme Pollock, Barry Richards and Clive Rice also effectively served life bans for being, well, South African, and the program understood that. I know that, fundamentally, the tour was morally corrupt, but for an innocent nine-year-old devoid of political allegiances and beliefs, it was simply a chance to see the world's best cricketers entertaining us on the world's most beautiful stage. Ultimately, I'd also like to believe that South African cricketers and administrators were trying to show the government the way forward, and that unity through sport was possible.

Summary:
"""

In [ ]:
with torch.no_grad():
  token_ids = tokenizer.encode(prompt, add_special_tokens=False ,return_tensors="pt")
  output_ids = model.generate(
      token_ids.to(model.device),
      max_new_tokens=512,
      do_sample=True,
      temperature = 0.1
  )

output = tokenizer.decode(output_ids[0][token_ids.size(1) :])
print(output)

In [ ]:
prompt = """
Summarize the following article in few sentences.
Text: My mom was always sports mad -- my dad not so much. But both my parents understood the importance of significant events, so when opportunity knocked, my parents opened up all doors for me. I grew up knowing full well that my country wasn't allowed to participate in international sports following the sporting boycott of 1977 which had been placed on South Africa in a protest against apartheid -- a system of racial segregation enforced by the ruling white minority. So when the South African rugby team toured New Zealand in 1981, my parents made sure that I was awoken at 04:30 to watch all three Test matches. I got to snuggle between my parents -- and a first -- they put the television in their bedroom. It was the ""flour bomb"" Test -- where a light aircraft flew over Auckland's Eden Park before and during the match, dropping flour bombs, smoke bombs and anti-apartheid leaflets -- where I began to realize that other countries didn't like us. I didn't understand why at the time, and it hurt a little. Later that day, I'll never forget my parents telling me that it could be a long time before I witnessed something like this again. I was six at the time. They were right; I would next see these two teams play as a 17-year-old. First trip . My first trip to Cape Town's Newlands Cricket Ground was to watch Western Province play a domestic match. It must have been my mom's first trip too, because I distinctly remember entering the ground and ending up sitting on the grass embankment, which was reserved for colored people. Confused, we sat down. A kind couple offered us some fruit, so we stayed. I was blissfully unaware of how significant this possibly was. We returned a couple of years later to watch my first ""Test"" match: South Africa vs. the West Indies. This time we were seated in the correct place, in the Oaks Enclosure, which ironically these days is a grass embankment. I remember the hype and the excitement about this special day. I had seen the Windies in their warmup game against Western Province and I knew that they were something special. They had vicious fast bowlers and flamboyant batsmen. We got there early, along with thousands of others, just to watch the warmups. The excitement was tangible. I don't remember much about the games -- I remember more about the players. They were heroes to us. Perhaps there was some curiosity too, something about the islanders from far, far away on our shores. We used to follow them around the boundaries of the ground, hunting their prized autographs. And their accents -- just too cool. Didn't see color . I know I didn't see the color. I don't think most kids did -- they were just cricketers. At the end of play we would run on the field to try to touch them and grab one more autograph. Somewhere there is footage of me congratulating Sylvester Clarke by patting his back. I remember telling all my friends that I had touched the great bowler. While I had forgotten about it, listening to Franklyn Stephenson speak reminded me of the time we were trying to get him to drink from some kid's Coke. He obliged and we all simply went crazy. I realize the significance of that event now. Imagine a black guy drinking from the same can? Perhaps our parents would have flipped out, but this wasn't about color to us. Other memories include how obliging and friendly they were. Happy to give autographs and happy to interact positively with the huge crowds. Branded a rebel . It was with incredible interest and fascination that I watched ""Branded a rebel"" -- a CNN World Sport documentary which first screened last year. It gave me greater insight into the players, the tour, West Indies cricket and indeed my home country. And while ""Branded a rebel"" is an appropriate title for the program, it could have just as easily been titled ""Branded Heroes."" That's what they were to us white South Africans, and I think deep down the Windies players felt that too. It's all about what side of the line you were sitting, I guess. On their return home, the West Indian players were shunned by society and some were forced to move abroad to make a living. The poverty and other such ills which befell some of their players really tugged at all of my emotions. I started thinking what we as South Africans could have done for them. Could we have done more? Did we look after them well enough after the fact? It's a difficult one. These men didn't deserve it, no matter what choices they made. Society can be cruel. Strangled by hatred . It eats me up to think what those players endured upon their return. They were strangled by such hatred in their home countries, and yet I wish they somehow knew what they had done for young cricket fans like me. I got to see ""real"" international cricket played at the ""highest"" level. I got to meet new cricketing greats. I got to bowl like Clarke, Collis King and Colin Croft. I got to emulate David Murray's great wicketkeeping skills on Clifton beach, field like Alvin Kallicharan and admire the patience and masterful batting of the great Lawrence Rowe. And something else -- they taught us not to take ourselves too seriously by showing how much one can enjoy themselves on the sports field. Their smiles, I'll never forget their smiles. Each of their names is entrenched in my memory, and of all of those who saw the matches. They became part of my upbringing and that of thousands of South African fans. What I didn't understand at the time, was what it took for those men to actually land on our shores (money aside). They must have been scared -- really scared. I know I would have been. In the interview with Clive Lloyd, longtime captain of the official West Indies team that dominated world cricket for two decades, he said no amount of money would have made him support a racist regime. I would go as far as to say no amount of money would have put my fears at ease to travel to South Africa. Courage . It's not for me to judge, but those men were courageous. In their cricket and in their collective decisions, and I respect that. A few years later, I did wonder how the black West Indian players got to travel using ""white"" transport and stay in ""white"" areas. I remember asking my mom how that worked. She didn't know, but you could see that my political curiosity was growing. I was disappointed that more players didn't take up the opportunity of being interviewed for the documentary. I was intrigued by Stephenson, King and Murray's thoughts, motives and reflective answers. There seemed no real regret, which was fascinating and pleasing for some reason. As for those who declined an interview, their wounds obviously cut deep. Stephenson was just how I remember him. Boisterous, bubbly and always looking on the bright side of life. We got to know him well here, as he returned to play for the Orange Free State. The fact that he still has all his memorabilia means that the tour meant something to him, and that he feels part of change and change of attitudes in South Africa. I don't know if they changed us as a society but I'd like to thing so, even if it was an indescribably fractured and dysfunctional society. And by this I mean that white South Africa packed out every cricket ground to watch black West Indies, went home and made sure that ""The Cosby Show"" was the most watched program on television -- and yet black people were pretty much denied and excluded from everything. Heart bleeds . ""Branded a rebel"" showed balanced reporting. I like the fact that you saw the tour from a South African perspective as well. Great players like Graeme Pollock, Barry Richards and Clive Rice also effectively served life bans for being, well, South African, and the program understood that. I know that, fundamentally, the tour was morally corrupt, but for an innocent nine-year-old devoid of political allegiances and beliefs, it was simply a chance to see the world's best cricketers entertaining us on the world's most beautiful stage. Ultimately, I'd also like to believe that South African cricketers and administrators were trying to show the government the way forward, and that unity through sport was possible.

Summary:
"""

In [ ]:
with torch.no_grad():
  token_ids = tokenizer.encode(prompt, add_special_tokens=False ,return_tensors="pt")
  output_ids = model.generate(
      token_ids.to(model.device),
      max_new_tokens=512,
      do_sample=True,
      temperature = 0.1
  )

output = tokenizer.decode(output_ids[0][token_ids.size(1) :])
print(output)

In [ ]:
prompt = """
Generate a short summary from this following text.
Text: Note that in the prompt above we didn't provide the model with any examples of text alongside their classifications, the LLM already understands "sentiment" -- that's the zero-shot capabilities at work. Instruction tuning has shown to improve zero-shot learning Wei et al. (2022). Instruction tuning is essentially the concept of finetuning models on datasets described via instructions. Furthermore, RLHF (reinforcement learning from human feedback) has been adopted to scale instruction tuning wherein the model is aligned to better fit human preferences. This recent development powers models like ChatGPT. We will discuss all these approaches and methods in upcoming sections. When zero-shot doesn't work, it's recommended to provide demonstrations or examples in the prompt which leads to few-shot prompting. In the next section, we demonstrate few-shot prompting.

Summary:
"""

In [ ]:
with torch.no_grad():
  token_ids = tokenizer.encode(prompt, add_special_tokens=False ,return_tensors="pt")
  output_ids = model.generate(
      token_ids.to(model.device),
      max_new_tokens=512,
      do_sample=True,
      temperature = 0.1
  )

output = tokenizer.decode(output_ids[0][token_ids.size(1) :])
print(output)

# **20 Examples**

In [ ]:
prompt = """
"Short Summary" means a summary which has only ten sentences with all the context of the text. Now generate a short summary from this following text.
Text: A Hero teacher who saved the life of a seriously ill pupil by giving her one of his kidney is to be honoured with an MBE. Ray Coe stepped in to rescue Alya Ahmed Ali, 13, after learning she was desperate for a donor. The 53-year-old father-of-one said he was left shocked and proud after learning he had been recognised in the Queen's New Year Honours list. Teacher Ray Coe, pictured left, is set to be awarded an MBE after he stepped in to rescue pupil, Alya Ahmed Ali, 13, pictured right, after learning she was desperate for a kidney donor . The special needs coordinator at Royal Docks Community School, Custom House, east London, has been awarded the gong for services to education and the community. He took part in the life-saving kidney transplant to rescue Alya, who suffers from a deadly condition called hydrocephalus, or water on the brain, which has led to severe learning difficulties. They have since become very close - and Mr Coe even spent part of Christmas with Alya and her grateful family. 'It was a bit of a shock to be made an MBE,' said modest Mr Coe. 'I was not expecting it at all. The special needs coordinator at Royal Docks Community School, Custom House, east London, pictured above with Alya,   has been awarded the gong for services to education and the community . Mr Coe, pictured with Alya and her father Ahmed Ali, said he was still a bit 'perplexed' and added: 'I don't see it as anything that no-one else would do' 'It's a very proud moment and there's a huge sense of honour.It seems like it's just spiralled. 'I never thought for a moment that it would become as big a story as it did. 'I'm still a bit perplexed, because I don't see it as anything that no-one else would do. 'For me the greatest thing to come out of it has been becoming a real part of Alya's family.' Mr Coe is a teacher at Royal Docks Community School in east London, where Alya is a pupil . Alya's dad, Ahmed Ali, 47, of Stratford, east London, said after Ray brought his daughter back from the brink: 'He has given Alya much more than just the gift of life.He's an amazing man, we owe him so much.' School head Wendy Bower also saluted Mr Coe, saying: 'Ray has gone above and beyond with the call of duty with this selfless and noble act. 'He's a very humble and modest man.The whole staff are in admiration for his kindness. 'He has given a new life to Alya and her whole family.' Summary: Teacher Ray Coe gave pupil Alya Ahmed Ali,13, one of his kidneys . The teenager  desperately  needed a donor after suffering from renal failure . The father-of-one, 53, has been awarded an MBE in the New Year Honours . He said he was not expecting it, but described it as a proud moment.

Short Summary:
"""


with torch.no_grad():
  token_ids = tokenizer.encode(prompt, add_special_tokens=False ,return_tensors="pt")
  output_ids = model.generate(
      token_ids.to(model.device),
      max_new_tokens=512,
      do_sample=True,
      temperature = 0.3
  )

output = tokenizer.decode(output_ids[0][token_ids.size(1) :])
print(output)

In [ ]:
prompt = """
"Short Summary" means a summary which has only ten sentences with all the context of the text. Now generate a short summary from this following text.
Text: (CNN) -- Palestinian journalists are finding reporting increasingly difficult in the face of the growing abuse toward them by Palestinian security forces in the West Bank and Gaza, according to a report released Wednesday by Human Rights Watch. The report, "No News is Good News: Abuses Against Journalists by Palestinian Security Forces," criticizes Palestinian Authority forces in the West Bank and Hamas security forces in Gaza, saying they are abusing local journalists and limiting press freedom in the Palestinian territories. The 35-page report focuses on seven incidents in the West Bank and two in Gaza, saying they are "cases in which security forces tortured, beat, and arbitrarily detained journalists, confiscated their equipment and barred them from leaving the West Bank and Gaza." Human Rights Watch cites a report issued by the Palestinian Center for Development and Media Freedom that says there was a 45% increase in the number of physical attacks, arrests, confiscation of equipment and limitation of movement in the year 2010. "We respect the Human Rights Watch report and other human rights organizations in what they do on the ground, and we have the readiness to investigate each case," said Maj. Gen. Adnan Dmeiri, spokesman for the Palestinian Authority security services. But, he said, "Why don't they consult us and ask us ... before publishing these reports?" He said the security services will investigate and "bring people to justice. The PA has no agenda to violate the rights of human beings and especially the rights of the journalists. We trust our justice system and the freedom of journalists is a priority of all things." Most of the abuses against journalists are connected to the rift between Fatah, which controls the West Bank's Palestinian Authority, and Hamas, which controls Gaza, according to the report. Pro-Hamas journalists are targeted by the Palestinian security forces in the West Bank and pro-Fatah journalists are targeted by the Hamas security in Gaza. The report stresses that the West Bank security forces have "virtual impunity for serious human rights violations" and states that the "utter failure of the (Palestinian Authority) leadership to address the prevailing culture of impunity for such abuses suggests that they reflect government policy." "We don't accept the comparison between us and the militia in Gaza," Dmeiri said. He said his organization has no political agenda and "we don't hold and detain people on their background. We detain people if they have criminal or security threats or matters that threaten the internal security." Large sums are donated to the Palestinian Authority security forces in the West Bank by the United States and the European Union. Human Rights Watch calls on the donor countries to condition their support to those security services by making sure that Palestinian Authority officials investigate and punish security personnel for their abuses. Attacks on International journalists in Gaza also have increased in the past couple of months. In March 2011, as journalists were covering a demonstration in Gaza calling for reconciliation between Fatah and Hamas, the offices of Reuters, CNN and NHK were raided by Hamas security forces. Efforts to get a response from Hamas officials were not immediately successful. The Tel Aviv-based Foreign Press Association released a statement expressing its "grave concern" that Hamas security forces "brutally attacked photographers and cameramen, beating them, breaking equipment and confiscating photos and video footage." The organization called the developments the latest in a "string of chilling attacks on reporters in Gaza." The United States and European Union have designated Hamas as a terrorist organization. Human Rights Watch describes itself on its website as "one of the world's leading independent organizations dedicated to defending and protecting human rights." CNN's Kareem Khadder contributed to this report. Summary: NEW: Palestinian security forces say they will investigate the allegations . The Human Rights Watch report focuses on seven incidents in the West Bank and two in Gaza . Journalists were "tortured, beat, and arbitrarily detained," the report says . The rift between Fatah and Hamas is behind most of the incidents, it says.

Short Summary:
"""


with torch.no_grad():
  token_ids = tokenizer.encode(prompt, add_special_tokens=False ,return_tensors="pt")
  output_ids = model.generate(
      token_ids.to(model.device),
      max_new_tokens=512,
      do_sample=True,
      temperature = 0.3
  )

output = tokenizer.decode(output_ids[0][token_ids.size(1) :])
print(output)

In [ ]:
prompt = """
"Short Summary" means a summary which has only ten sentences with all the context of the text. Now generate a short summary from this following text.
Text: WASHINGTON (CNN)  -- Terrorists are likely to use a weapon of mass destruction somewhere in the world in the next five years, a blue-ribbon panel assembled by Congress has concluded. Police watch over travelers at New York's Grand Central Terminal before Thanksgiving. They are more likely to use a biological weapon than a nuclear one -- and the results could be devastating, the chairman of the commission told CNN. "The consequences of a biological attack are almost beyond comprehension. It would be 9/11 times 10 or a hundred in terms of the number of people who would be killed," former Sen. Bob Graham said. He cited the flu virus that killed millions of people in 1918 as an example. "Today it is still in the laboratory, but if it should get out and into the hands of scientists who knew how to use it for a violent purpose, we could have multiple times the 40 million people who were killed 100 years ago," he said.  Watch how officials worry about a biological terror attack  » . The U.S. government "needs to move more aggressively to limit" the spread of biological weapons, the commission said in its report. Graham warned that such measures would be costly, but were necessary. "The leadership of this country and the world will have to decide how much of a priority ... they place on avoiding the worst weapons in the world getting in the hands of the worst people in the world," he said. "It is not going to be cheap. It is not going to be accomplished without some sacrifices. It won't be accomplished without putting this issue ahead of some other competing national and international goals. But I think our safety and security depend upon doing so," he added. Graham said a biological attack was more likely than a nuclear one because it would be easier to carry out. Biological weapons "are more available," he said. "Anthrax is a natural product of dead animals. Other serious pathogens are available in equally accessible forms." "There are so many scientists who have the skills to convert a pathogen from benign, helpful purposes into an illicit, very harmful weapon," he added. But the commission warned that there is also a threat of nuclear terrorism, both because more countries are developing nuclear weapons and because some existing nuclear powers are expanding their arsenals. "Terrorist organizations are intent on acquiring nuclear weapons," said the report, which was published Tuesday on the Internet and will be officially released Wednesday. CNN obtained a copy of the report Monday evening. It cited testimony before the commission from former Sen. Sam Nunn, who said that the "risk of a nuclear weapon being used today is growing, not receding." The report recommends a range of measures, including increased security and awareness at biological research labs and strengthening international treaties against the spread of biological and nuclear weapons. "Many biological pathogens and nuclear materials around the world are poorly secured -- and thus vulnerable to theft by those who would put these materials to harmful use, or would sell them on the black market to potential terrorists," the report warned. The commission expressed particular concern about the nuclear programs of Iran and North Korea, and about Pakistan, which it described as "the intersection of nuclear weapons and terrorism." While observing that Pakistan is a U.S. ally, the report said, "the next terrorist attack against the United States is likely to originate from within the Federally Administered Tribal Areas" in Pakistan. The tribal areas lie in northwest Pakistan where the government exerts little control; the United States says it is a haven for militants from both Pakistan and neighboring Afghanistan. Congress created the commission to investigate and report on WMD and terrorism in line with a recommendation from the 9/11 Commission, which compiled a report on the September 11, 2001, terrorist attacks on the United States. Commissioners heard testimony from more than 250 experts from around the world over the course of their six-month investigation. CNN's Jeanne Meserve contributed to this report. Summary: NEW: Next terror attack on U.S. likely to originate in Pakistan tribal areas, report says. Terrorists with biological weapons could kill millions, panel's chairman says. Biological attack more likely than nuclear attack, report says. Number of nations with nuclear weapons also growing, panel says.

Short Summary:
"""


with torch.no_grad():
  token_ids = tokenizer.encode(prompt, add_special_tokens=False ,return_tensors="pt")
  output_ids = model.generate(
      token_ids.to(model.device),
      max_new_tokens=512,
      do_sample=True,
      temperature = 0.3
  )

output = tokenizer.decode(output_ids[0][token_ids.size(1) :])
print(output)

In [ ]:
prompt = """
Generate a short summary of ten sentences from this following Text.
Text: The U.S. Supreme Court on Tuesday refused to stop Texas from implementing a part of a new abortion law that requires doctors to get admitting privileges at a hospital within 30 miles of the clinic where they're providing abortion services. A federal appeals court had reinstated the key part of the law, which is considered among the most restrictive in the country. Abortion-rights groups, which say more than a third of centers that had provided abortions in Texas have since stopped, then asked the Supreme Court to overturn the appeals court ruling. "We may not vacate a stay entered by a court of appeals unless that court clearly and 'demonstrably' erred in its application of 'accepted standards,'" Justice Antonin Scalia wrote in support of Tuesday's order. "Reasonable minds can perhaps disagree about whether the Court of Appeals should have granted a stay in this case. But there is no doubt that the applicants have not carried their heavy burden of showing that doing so was a clear violation of accepted legal standards -- which do not include a special 'status quo' standard for laws affecting abortion," he wrote. Four justices -- Stephen Breyer, Ruth Bader Ginsburg, Sonia Sotomayor and Elena Kagan -- dissented. Justices dismiss state law restricting medical abortions . The original lawsuit, filed in U.S. District Court in Austin by Planned Parenthood on behalf of more than a dozen women's health care providers across Texas, alleged the state's new abortion law violates the constitutional rights of women and puts unreasonable demands on doctors who perform abortions. The bill originally failed to gain approval because of a Democratic filibuster led by state Sen. Wendy Davis. Gov. Rick Perry then called the Legislature into a second special session to continue consideration of the bill. Davis, who is running for governor, blasted Tuesday's decision. "Clinics will close and women's health will be hurt because of this law. This is an abuse of power by politicians in Austin. I trust women to make their own decisions and will continue to work to make sure that women and mothers are safe and have access to adequate health care," she said in a statement. Perry cheered the ruling. "This is good news both for the unborn and for the women of Texas, who are now better protected from shoddy abortion providers operating in dangerous conditions. As always, Texas will continue doing everything we can to protect the culture of life in our state," he said. Supreme Court justices dismiss another Oklahoma law on abortion . Summary: Four U.S. Supreme Court justices dissent in the ruling .
Wendy Davis: "This is an abuse of power by politicians". Texas governor vows to protect the "culture of life in our state"


Short Summary:
"""


with torch.no_grad():
  token_ids = tokenizer.encode(prompt, add_special_tokens=False ,return_tensors="pt")
  output_ids = model.generate(
      token_ids.to(model.device),
      max_new_tokens=512,
      do_sample=True,
      temperature = 0.7
  )

output = tokenizer.decode(output_ids[0][token_ids.size(1) :])
print(output)

In [ ]:
prompt = """
"Short Summary" means a summary which has only ten sentences with all the context of the text. Now generate a short summary from this following text.
Text: (CNN) -- Tour de France race director Christian Prudhomme has confirmed the 2013 race -- the centenary edition -- will start on the French island of Corsica. Corsica, which lies off the southern French coast, will host the first three stages of the race from Porto-Vecchio to Calvi, the first time the Tour has ever visited. Prudhomme told AFP: "For the hundredth edition of the race, we wanted to go somewhere new, somewhere spectacular. "In Corsica, we have all of that. We have everything in place for an enchanting start to the race," he added. The opening 201.5km stage from Porto-Vecchio to Bastia is likely to favor the sprinters, while the 153km second stage from Bastia to Ajaccio, is expected to see the climbers come to the fore. The third stage is relatively short, just 145km from Ajaccio to Calvi. Every French region has hosted a stage of the Tour but the race has never previously crossed over to Corsica, the birthplace of Napoleon Bonaparte, for logistical reasons. The 2012 Tour de France will begin in the Belgian city of Liege on June 30. Summary: The 2013 Tour de France will start from the French island of Corsica .
Three stages will be held on the island, the first time the Tour has visited .
2013 marks the 100th edition of the biggest cycle race in the world .


Short Summary:
"""


with torch.no_grad():
  token_ids = tokenizer.encode(prompt, add_special_tokens=False ,return_tensors="pt")
  output_ids = model.generate(
      token_ids.to(model.device),
      max_new_tokens=512,
      do_sample=True,
      temperature = 0.3
  )

output = tokenizer.decode(output_ids[0][token_ids.size(1) :])
print(output)

In [ ]:
prompt = """
"Short Summary" means a summary which has only ten sentences with all the context of the text. Now generate a short summary from this following text.
Text: ISLAMABAD, Pakistan (CNN) -- Shazia knows what she sees when she looks in the mirror: a woman, feminine, delicate, attracted to beauty. But when others look at Shazia they see something else: a misfit. Shazia dances for the cameras at a party. "People start staring at me in such a manner as if an animal from the jungle has come into the city." Shazia says. Shazia belongs to Pakistan's community of eunuchs. It is a term here that loosely refers to transsexuals, transvestites or hermaphrodites, like Shazia, who are born with both male and female sex organs. For Shazia what nature created, has made a life of hell. "I get so confused. I feel very strange that Allah could have made me a boy, or he could have made me a girl, but this way, neither boy nor girl. This life is very strange. It is a bad life," Shazia said. "I do not want to accept this life. Had it been legal to commit suicide I would have done it." When she was a child, Shazia said she had no doubt that she felt more a girl than a boy.  Watch Shazia handle stares from men  » . "When I used to go to school there used to be benches for boys on one side and girls on the other; my teacher always made me sit with the girls." she said. Shazia left home when she was in her early teens. She said it was to save her family embarrassment and shame. Like other eunuchs in Pakistan, she has lived in colonies. They are often shunned by their families and forced to make a living by begging or as prostitutes. The colonies can be brutal places, where young people are often beaten and exploited. When Shazia was 17, she says she was drugged and the head of the colony performed brutal surgery severing Shazia's penis. But Shazia has survived the taunts, the humiliation and the savagery; now she is fighting back. Shazia has formed an action group and is campaigning for eunuchs like her to get proper recognition. Right now her state identity card lists her as a male. She wants to be counted as a whole person. Shazia has found a champion, human rights lawyer, Muhammad Aslan Khaki. Dr. Khaki presented a petition to the Supreme Court, leading to the first register of eunuchs in Pakistan. The court has ordered Pakistan's government departments to carry out a survey and recommend measures to fully integrate eunuchs into society without prejudice. In earlier centuries eunuchs were seen as having the power to bring good luck. For Shazia and others, their luck may finally be changing. Summary: Pakistan's eunuch community campaigns for proper recognition .
In Pakistan the term covers transsexuals, transvestites, hermaphrodites .
Shazia, a hermaphrodite, tells how her penis was removed at 17 .
She says she feels she is a woman, but her identity card calls her male .


Short Summary:
"""


with torch.no_grad():
  token_ids = tokenizer.encode(prompt, add_special_tokens=False ,return_tensors="pt")
  output_ids = model.generate(
      token_ids.to(model.device),
      max_new_tokens=512,
      do_sample=True,
      temperature = 0.3
  )

output = tokenizer.decode(output_ids[0][token_ids.size(1) :])
print(output)

In [ ]:
prompt = """
Generate a short summary from this following text.
Text: By . Harriet Hernando . An elderly man was filmed riding his mobility scooter down a street â€“ even though it was flooded and had turned into a river. While cars lay abandoned in the road, the unnamed pensioner ploughed on regardless, much to the hilarity of the bystanders filming him. The incredible video was shot in Arnhem, Netherlands, when it was blighted with flooding after the river Rhine burst its banks. The low-lying Netherlands has been fighting back water for more than a thousand years when farmers first built dikes to protect their land. Now a complicated network of canals and pumping stations cover the country keeping it high and dry. But as this video shows, a bit of flooding is nothing the Dutch canâ€™t handle. The news comes as a British pensioner caused chaos on the M1 motorway by riding his scooter the wrong way down the hard shoulder. The 85-year-old escaped unharmed after being escorted home by the police. Making waves: Pensioner drives down the flooded road where others were forced to abandon their cars in Arnhem, Netherlands . Summary: The elderly man drives down the road where cars lie abandoned .
He shows that a bit of flooding is nothing the Dutch can't handle .
Is filmed by amused onlookers as he battles through the floods .


Short Summary:
"""


with torch.no_grad():
  token_ids = tokenizer.encode(prompt, add_special_tokens=False ,return_tensors="pt")
  output_ids = model.generate(
      token_ids.to(model.device),
      max_new_tokens=512,
      do_sample=True,
      temperature = 0.3
  )

output = tokenizer.decode(output_ids[0][token_ids.size(1) :])
print(output)

In [ ]:
prompt = """
"Short Summary" means a summary which has only ten sentences with all the context of the text. Now generate a short summary from this following text.
Text: By . Jessica Jerreat . A Minnesota air stewardess has finally welcomed home her three adopted daughters seven years after she first met the girls in a Guatemalan orphanage. Suann Hibbs met her daughters in 2008, but when the Guatemalan government cut back on foreign adoptions the family were left in legal limbo. It took seven years, 30 trips abroad and a lot of help from Senator Amy Klobuchar before officials finally allowed Ms Hibbs to take the three sisters home last week. Scroll down for video . Mother's love: Suann Hibbs hugs one of her three adopted daughters . It is a dream come true for Ms Hibbs who had been keen to become a mother for years. 'I still feel like this is so surreal. I . mean, I just have to pinch myself. When they come in, in the morning . and crawl in bed with me. I just have goose bumps thinking about it,' she told CBS News. When she first met the children, twins Savanna and Sophia were one-year-olds and their sister Sydney was a newborn. 'I just fell in love with them as soon as I saw them. I just thought morally, I just can't let them go,' Ms Hibbs, who is not married and found the girls through a church adoption group, said. The adoption agency Ms Hibbs had used estimated that within six to nine months the paperwork would be completed and the girls would be with her in Edina, Minneapolis. But the process was massively delayed as developing countries including Guatemala started to restrict foreign adoptions while they tackled problems with fraud and child trafficking. Family: Twins Savanna and Sophia were a year old and their little sister Sydney was a newborn when Suann first met them . Growing up: By the time the sisters finally arrived at their new home the twins were 8 and Sydney was 7 . In November 2007, her adoption case was put on hold during a country-wide investigation, and it looked as though she may lose her children. While the case was being slowly processed the three sisters were moved between several orphanages and foster homes. The constant moving made it hard for Ms Hibbs to keep track of them, and also meant that the girls have not had any formal education. 'They needed a permanent home,' Ms Klobuchar, who met Guatemalan officials on Ms Hibbs' behalf, said. Settling in: After living in several orphanages the sisters finally have a permanent place to call home . 'It took a little time, but . they got to the right place and allowed these children to be adopted.' She added: 'A lot of times when you personally meet . with the ambassador like I did and you tell the story of these three . little girls and this mother who has been trying for seven years, they . listen.' Although the girls, now aged eight and seven, speak only a little English, they are settling into their new home well. They will start school in a couple of weeks and attend summer classes to help them catch up with their new classmates. Summary: Suann Hibbs was left in legal limbo after Guatemala put hold on foreign adoptions .
Air stewardess turned to senator for help to bring the sisters back to Minneapolis .


Short Summary:
"""


with torch.no_grad():
  token_ids = tokenizer.encode(prompt, add_special_tokens=False ,return_tensors="pt")
  output_ids = model.generate(
      token_ids.to(model.device),
      max_new_tokens=512,
      do_sample=True,
      temperature = 0.3
  )

output = tokenizer.decode(output_ids[0][token_ids.size(1) :])
print(output)

In [ ]:
prompt = """
"Short Summary" means a summary which has only ten sentences with all the context of the text. Now generate a short summary from this following text.
Text: Paris (CNN) -- French officials Tuesday condemned an Israeli airstrike on Gaza that wounded its consul, his wife and their daughter, calling on Israel to avoid civilian casualties. "France strongly deplores the consequences of this air strike," a statement from the Ministry of Foreign Affairs in Paris announced. "While France is committed to security in Israel, it reaffirms the imperative need to avoid attacks on civilians. This imperative was reaffirmed to the Israeli authorities." The consul, Majdi Shakoura, was at home with his family in the northern end of Gaza when the airstrike hit about 200 meters (650 feet) away, the Foreign Ministry said. The strike blew out their windows, and they were struck by shards of flying glass, the ministry said. Capt. Aryeh Shalikar, an Israeli military spokesman, told CNN the airstrike was aimed at Palestinian militants who fired a rocket into southern Israel late Sunday. Shalikar said the Israel Defense Forces "never received any official statement from any source" regarding injuries to Shakoura or his relatives. "The IDF wishes to convey that missiles are being fired at Israeli civilians from terrorists, and it has no intention of harming civilians when it returns fire at terrorists," he said. Palestinian security and medical officials said one person died and several others were wounded in the airstrike, which they said struck a Hamas naval building in northern Gaza. A Hamas security official said two Israeli rockets struck the building. France has a consulate and a cultural center in Gaza "to support the population," the Foreign Ministry said. France has helped build water and sewer systems and rebuild hospitals in Gaza, which is ruled by the Islamic movement Hamas. The United States, European Union and Israel classify Hamas as a terrorist organization, and Israel has maintained a controversial blockade of Gaza since Hamas took power in 2007. CNN's Naima Benallal in Paris and Izzy Lemberg in Jerusalem contributed to this report. Summary: NEW: French consul, family hit by flying glass, ministry says .
France calls on Israel to avoid civilian casualties .
Israel says the strike was in response to Palestinian rocket fire .


Short Summary:
"""


with torch.no_grad():
  token_ids = tokenizer.encode(prompt, add_special_tokens=False ,return_tensors="pt")
  output_ids = model.generate(
      token_ids.to(model.device),
      max_new_tokens=512,
      do_sample=True,
      temperature = 0.3
  )

output = tokenizer.decode(output_ids[0][token_ids.size(1) :])
print(output)

In [ ]:
prompt = """
"Short Summary" means a summary which has only ten sentences with all the context of the text. Now generate a short summary from this following text.
Text: (CNN)Egyptian President Abdel Fattah el-Sisi has called for a "religious revolution," asking Muslim leaders to help in the fight against extremism. In a speech celebrating the birthday of the Prophet Muá¸¥ammad, which coincided with New Year's Day, he said they had no time to lose. "I say and repeat, again, that we are in need of a religious revolution. You imams are responsible before Allah. The entire world is waiting on you. The entire world is waiting for your word ... because the Islamic world is being torn, it is being destroyed, it is being lost. And it is being lost by our own hands," el-Sisi said. "We need a revolution of the self, a revolution of consciousness and ethics to rebuild the Egyptian person -- a person that our country will need in the near future," the President said. El-Sisi, himself a pious man, was elected in May after leaving the military to run for the office. A former defense minister, he led the ouster of Mohamed Morsy -- the Islamist who was Egypt's first democratically elected President -- and has long positioned himself as a more secular option, and defender against extremist views. "It's inconceivable that the thinking that we hold most sacred should cause the entire Islamic world to be a source of anxiety, danger, killing and destruction for the rest of the world.  Impossible that this thinking -- and I am not saying the religion -- I am saying this thinking," el-Sisi said. He continued: "This is antagonizing the entire world. It's antagonizing the entire world! Does this mean that 1.6 billion people (Muslims) should want to kill the rest of the world's inhabitants -- that is 7 billion -- so that they themselves may live? Impossible!â€‹" While el-Sisi's speech included some powerful language, H.A. Hellyer, a nonresident fellow at the Center for Middle East Policy at the Brookings Institution and research associate at the Kennedy School of Government at Harvard University, said the President has made similar statements in the past. "There is little to suggest (el-Sisi) is interested in some sort of Lutheran reformation of Islam. By all accounts, he's quite comfortable with the prevailing leadership of the Azhari establishment. "If anything, he wants to empower it further in order to push forward a counternarrative against radical Islamism. The real question is: How credible can such a state-empowered counternarrative be?" Hellyer said. On Tuesday, the President visited the main Coptic cathedral in Cairo to attend a Christmas mass and make a short speech. He is the first president to attend such a mass since the revolution. "We will build our country together. We will accommodate each other. We will love each other," el-Sisi said in that speech. Summary: President Abdel Fattah el-Sisi speaks to celebrate the birthday of the Prophet Muá¸¥ammad .
"We need a revolution of the self, a revolution of consciousness," he says .


Short Summary:
"""


with torch.no_grad():
  token_ids = tokenizer.encode(prompt, add_special_tokens=False ,return_tensors="pt")
  output_ids = model.generate(
      token_ids.to(model.device),
      max_new_tokens=512,
      do_sample=True,
      temperature = 0.7
  )

output = tokenizer.decode(output_ids[0][token_ids.size(1) :])
print(output)

In [ ]:
prompt = """
"Short Summary" means a summary which has only ten sentences with all the context of the text. Now generate a short summary from this following text.
Text: (CNN) -- A 24-year-old woman in a hospital bed fighting off flesh-eating bacteria has to be told repeatedly -- each time she wakes up -- what has happened, her parents told CNN on Monday. The medication Aimee Copeland is given leads her to forget each time she falls asleep. "It's scary to her," said her mother, Donna Copeland. She asks where she is and "doesn't understand." Yet Aimee Copeland -- who has lost a leg and part of her abdomen to the virulent bacteria and may lose more, including her fingers -- is keeping her spirits strong, her father said. Can I get this flesh-eating bacteria? "We really don't see the suffering side of it. We see the miraculous survival," Andy Copeland said. "I think that's the story that's inspired us, that's the story that's inspired, I think, the nation at this point." On Facebook, he wrote that doctors have used words like "astonishing," "confounding" and "mind-boggling" to describe the young woman's recovery. The master's student in psychology at the University of West Georgia was out with friends on May 1 near the Little Tallapoosa River, about 50 miles west of Atlanta, when she grabbed onto a homemade zip line. It snapped. The accident left her with a gash in her left calf that took 22 staples to close. Three days later, when the pain continued, a friend took her to an emergency room, where she was diagnosed with necrotizing fasciitis and flown to Augusta for surgery. She had contracted the flesh-devouring Aeromonas hydrophila. The bacterium is "remarkably common in the water and in the environment," according to Dr. Buddy Creech, an assistant professor of pediatric infectious diseases at Vanderbilt University. "When it gets into those deeper tissues, it has a remarkable ability to destroy the tissues that surround it in sort of this hunt for nutrition," he said. "When it does that, those tissues die, and you see the inflammation and the swelling and the destruction that can be very difficult to control." In most cases, people contract the bacteria by swallowing them, resulting in diarrhea. Aimee Copeland's case was much more rare. Her wound became infected, "and the infection (ran) wild," Creech said. A blog set up by the University of West Georgia psychology department said Aimee Copeland will suffer the loss of her fingers. "However, physicians have hope of bringing life back to the palms of her hands, which could allow her the muscle control to use helpful prosthetics. They are awaiting a safe time before embarking on surgery for this." Speaking to CNN on Monday, her father said doctors were assessing "day by day, or even hour by hour." Copeland has told his daughter that one day, the family will celebrate Aimee Day -- when she will be able to breathe on her own. "We're going to celebrate that day forever for the rest of your life," he told her. "It's the day that my daughter was delivered from this horrible, horrible disease." If there's anything to be learned, Andy Copeland said, it's not to use homemade zip lines. Aimee Copeland's parents say that when she wakes up, she expresses concern about finishing her thesis. In her studies, she has focused on eco-psychology -- the idea that harmonizing with nature can be a powerful tool in ensuring one's psychological health and vitality. CNN's Josh Levs, Greg Botelho and Brooke Baldwin contributed to this report. Summary: The medication Aimee Copeland is given leads her to forget what has happened .
She may lose her fingers .
Doctors have already amputated one of her legs and part of her abdomen .
Her father promises they will one day celebrate her ability to breathe on her own .


Short Summary:
"""


with torch.no_grad():
  token_ids = tokenizer.encode(prompt, add_special_tokens=False ,return_tensors="pt")
  output_ids = model.generate(
      token_ids.to(model.device),
      max_new_tokens=512,
      do_sample=True,
      temperature = 0.3
  )

output = tokenizer.decode(output_ids[0][token_ids.size(1) :])
print(output)

In [ ]:
prompt = """
"Short Summary" means a summary which has only ten sentences with all the context of the text. Now generate a short summary from this following text.
Text: A prankster dad may win the award for cruelest joke after he tricked his wife into thinking he threw his toddler son off of the balcony. Roman Atwood set up cameras and enlisted the help of his two sons in an elaborate attempt to frighten his unassuming wife and fool her into thinking he'd accidentally killed their toddler son. The stunt, which was posted on YouTube on November 30 and already has almost 5 million views, begins with Atwood playing with his son, who is dressed in a Spider-Man costume covering his face, on the top floor of their Los Angeles, California home. Like any other day: A loving mother returns to her home in Los Angeles where she finds her husband Roman Atwood playing with their toddler son who is dressed in a Spider-Man costume . The switcheroo: Roman Atwood enlists the help of his elder son who leads the toddler into the other room then hands his dad the small dummy dressed in the same costume as his son . The launch: Roman Atwood pretends to throw his son over the balcony as  his wife lets out a shriek and rushes towards what she believes is her fallen son . Atwood and his son are greeted lovingly by the boyâ€™s mother, who is then sent downstairs 'to grab them some water' so that they can prepare the prank behind her back. Atwood enlists the help of his older son who leads his younger brother into the other room them substitutes the toddler for a dummy, which Atwood then tosses over the railing. His wife runs downstairs and screams in paralyzing fear as she sees the boy falling to the ground below and laying motionless on the ground as though he had died. She unveils the boy's mask to reveal a dummy under the costume and then proceeds to shoot a disgusted glance at her husband. 'Why would you do that?!' she screams still trembling in shock. 'It's just for fun,' Atwood responds as he shrugs his shoulders . She then sentences her husband to a lonely night of sleeping on the couch and gives him a little kick in the groin before she storms out of the room. Petrified: The concerned mother rushes towards the dummy she believes to be her badly injured son . Unmasked: the mother removes the mask to see the damage done and sees  what she thought was her son is nothing more than a dummy . Horrified: Atwood's wife tells him that he'll be sleeping on the couch and gives him a kick to the groin before leaving the room in a huff . Summary: Roman Atwood set up cameras and enlisted the help of his two sons in an elaborate attempt to frighten his unassuming wife .
Atwood pretends to throw their son, dressed in a Spider-Man suit, over the balcony by using a  dummy  dressed in the same costume .
His wife was less than pleased and told him that he'd be sleeping on the couch .


Short Summary:
"""


with torch.no_grad():
  token_ids = tokenizer.encode(prompt, add_special_tokens=False ,return_tensors="pt")
  output_ids = model.generate(
      token_ids.to(model.device),
      max_new_tokens=512,
      do_sample=True,
      temperature = 0.3
  )

output = tokenizer.decode(output_ids[0][token_ids.size(1) :])
print(output)

In [ ]:
prompt = """
"Short Summary" means a summary which has only ten sentences with all the context of the text. Now generate a short summary from this following text.
Text: John McCain's office is denying reports that Syrian rebels the senator was pictured with were responsible for the kidnap of Lebanese Shi'ites. According to the Lebanese press, two of the men pictured with Mr McCain had taken eleven people from Lebanon hostage. Nine of the pilgrims are still being held a year after they were taken hostage as they returned to Beirut from a pilgrimage in Iran. Rebel meeting: Senator John McCain has been accused of posing with members of Northern Storm . As negotiations continue for the release of the remaining hostages, the Daily Star in Beirut claims to have identified two of the men pictured with Mr McCain as being with the Northern Storm group believed to be behind the kidnapping. Mr McCain's spokesman Brian Rogers told  Buzzfeed: 'A number of the Syrians who greeted . Senator McCain upon his arrival in Syria asked to take pictures with . him, and as always, the Senator complied.' He added it 'would be regrettable' if any of the men turned out to be the suspected kidnappers, Mohamed Nour and Abu Ibrahim. The senator's spokesman added that Mr McCain did not condone 'the kidnapping of Lebanese Shia pilgrims . or [have] any communication with those responsible'. Secret visit: John McCain made a brief stop in Syria to meet leaders of the rebel opposition on Monday . Anwar Ibrahim, one . of the men taken hostage by the Northern Storm group, claims he recognized . Mohammad Nour, chief spokesman and photographer for the kidnappers, . standing next to Mr McCain. 'I recognized him immediately. He was the photographer who was brought in to take our photos. He works with the kidnappers. He knows them very well,' Mr Ibrahim told the Daily Star. He also claims to have recognized another man associated with the hostage-takers. However, Mr Rogers said none of the people Mr McCain met on his brief trip to Syria on Monday shared the names of the kidnappers. 'As the Syrian Emergency Task Force has . said: "Senator McCain did not go to Syria to meet with anyone named . Mohamad Nour or Abu Ibrahim. Two members of our organization were . present in the meeting, and no one called himself by either name",' he added. The senator's spokesman suggested the story could be part of the Assad-regime's propaganda, adding: 'Itâ€™s not surprising that the . pro-Hezbollah forces supporting the Assad regime who originally promoted . this distortion would seek to smear anything and anyone supporting the . Syrian opposition.' Secret visit: Senator McCain poses with Syrians during his meeting with the rebel opposition fighting Assad . Mr McCain traveled to Syria to meet leaders of the rebel opposition, who have asked the U.S for weapons and airstrikes against Hezbollah and Assad's forces. Since the civil war began after peaceful protests were met with violence about two years ago, at least 80,000 people have been killed. Mr McCain has been critical of President Obama for not taking enough action in the crisis, according to Politico. Summary: Northern Storm members pictured with McCain, Lebanon claims .
Kidnappers have been holding nine pilgrims hostage for a year .


Short Summary:
"""


with torch.no_grad():
  token_ids = tokenizer.encode(prompt, add_special_tokens=False ,return_tensors="pt")
  output_ids = model.generate(
      token_ids.to(model.device),
      max_new_tokens=512,
      do_sample=True,
      temperature = 0.3
  )

output = tokenizer.decode(output_ids[0][token_ids.size(1) :])
print(output)

In [ ]:
prompt = """
"Short Summary" means a summary which has only ten sentences with all the context of the text. Now generate a short summary from this following text.
Text: (CNN) -- Cyprus, the tiny Mediterranean island bailed out a year ago, remains "in difficulty...but not as bad as one would have expected," its finance minister Harris Georgiades told CNN during an interview in the capital city Nicosia. The country, which was given a 10 billion euro ($13.7 billion) injection by the European Union and IMF ago after its banking system collapsed, was the smallest of the euro economies to get help during the bloc's financial crisis. Unlike the Greek economy which has seen more than a quarter of its GDP wiped out during the past five years, the finance minister said that Cyprus will lose about half that. The European Commission is now forecasting growth in 2015 after a steep contraction of 4.8% this year and 6% percent last year. According to Georgiades, the situation is "bad and I have never tried to paint a rosy picture." The country has been "dealing with shortcomings and there were significant shortcomings, we have to admit." Cypriot banks were big buyers of Greek bonds during the pre-crisis boom, before the 2004 Athens Olympics and until the bottom fell out of economy in 2008. Cyprus, crippled by losses on Greek debt, set a precedent in the euro crisis by imposing losses on bank depositors. It was a move which infuriated savers, worried investors and pushed the euro crisis back into the headlines. But Georgiades said the country's main mistake was letting the crisis escalate. "We should have seen action both on behalf of the Cypriot authorities and the EU two years ago," he said. One of the key challenges during the rebuilding effort is dealing with an estimated $40 billion of non-performing loans which are holding back fresh lending. When asked if a "bad bank" could be set up to absorb those loans, Georgiades said the banking sector and supervisory authorities "need to work in a coordinated effort to deal with this problem," but added, "I would say that all the possibilities should be left on the table." Meanwhile, Cyprus' relationship with Russia is under pressure as the West increases economic pressure on Vladimir Putin's government and his allies. Relations with the West fractured after Russia's military intervention in Crimea and its amassing of troops on Ukraine's border. Russians are strategic investors in Cyprus, and wealthy individuals have long been drawn to the island and its low tax rates. A total of $30 billion has been invested by Russian companies over the past two decades, according to the Russian statistical agency ROSSTAT, putting Cyprus just behind the Netherlands as the top EU recipients. Georgiades said Cyprus -- along with its EU partners -- was supporting a "clear coherent common stance" and was not pushing toward a "new cold war." The country's stance is based on principles of "sovereign and human rights," Georgiades said. "We believe these principles are best served with well thought cool headed action and reaction and not a rushed move into a cold war mentality," he added. READ MORE: U.S. investors and the Cyprus mess . WATCH MORE: Could Cyprus be the next luxury yacht marina? Summary: Cyprus is starting to rebuild after its banking collapse a year ago, and 10 billion euro bailout .
The island nation's finance minister Harris Georgiades says the situation "is bad"
As the country tries to recover, it is also dealing with its relationship with Russia .
Russia, now being ostracized by the West, is a significant investors in Cyprus .


Short Summary:
"""


with torch.no_grad():
  token_ids = tokenizer.encode(prompt, add_special_tokens=False ,return_tensors="pt")
  output_ids = model.generate(
      token_ids.to(model.device),
      max_new_tokens=512,
      do_sample=True,
      temperature = 0.3
  )

output = tokenizer.decode(output_ids[0][token_ids.size(1) :])
print(output)

In [ ]:
prompt = """
Generate a short summary from this following text.
Text: (CNN) -- A Turkish television show is offering contestants what it claims is the "biggest prize ever" -- the chance for atheists to convert to one of the world's major religions. The TV show offers converts to Islam the chance to visit Mecca. The show, called "Tovbekarlar Yarisiyor," or "Penitents Compete," features a Muslim imam, a Catholic priest, a Jewish rabbi and a Buddhist monk attempting to persuade 10 atheists of the merits of their religion, according to CNN Turk. If they succeed, the contestants are rewarded with a pilgrimage to one of their chosen faith's most sacred sites -- Mecca for Muslims, Jerusalem for converts to Judaism, a trip to Tibet for Buddhists and the chance to visit Ephesus and the Vatican for Christians. Ahmet Ozdemir, deputy director of Turkish channel Kanal T, which will air the show from September, said the program aimed to "turn disbelievers on to God." "People are free to believe anything they want. Our program does not have a say," he said, according to Turkish newspaper Hurriyet. Contestants will be judged by a panel of eight theologians and religious experts prior to going on the show to make sure their lack of faith is genuine. But the show has been condemned by Turkish religious leaders. The head of the country's supreme council of religious affairs, Hamza Aktan, told CNN Turk that it was "disrespectful" to place different faiths in competition with each other and accused Kanal T of using religion to boost ratings. "To do such a thing for the sake of ratings, not only with Islam but with all religions is disrespectful," said Aktan. "Religion should not be the subject of this type of program." Although Turkey has a predominantly Muslim population and culture, religion is a sensitive subject because of the country's staunchly secular constitution which outlaws most displays of faith in public life. Last year the Islamist-influenced government of Turkish Prime Minister Recep Tayyip Erdogan clashed with the country's constitutional court when judges overturned the efforts of Erdogan's AK Party to lift a ban on female students wearing headscarves at public universities. Aylin Yazan at CNN Turk contributed to this story. Summary: Turkish TV show features imam, Catholic priest, Jewish rabbi, Buddhist monk .
Religious leaders attempt to persuade atheists to "convert" to their faith .
Show has prompted criticism from religious groups who say it is "disrespectful"


Short Summary:
"""


with torch.no_grad():
  token_ids = tokenizer.encode(prompt, add_special_tokens=False ,return_tensors="pt")
  output_ids = model.generate(
      token_ids.to(model.device),
      max_new_tokens=512,
      do_sample=True,
      temperature = 0.7
  )

output = tokenizer.decode(output_ids[0][token_ids.size(1) :])
print(output)

In [15]:
prompt = """
Give me a short summary of the following text. One hot day, a thirsty crow flew all over the fields looking for water. For a long time, he could not find any. He felt very weak, almost lost all hope. Suddenly, he saw a water jug below the tree. He flew straight down to see if there was any water inside. Yes, he could see some water inside the jug! The crow tried to push his head into the jug. Sadly, he found that the neck of the jug was too narrow. Then he tried to push the jug to tilt for the water to flow out, but the jug was too heavy. The crow thought hard for a while. Then, looking around it, he saw some pebbles. He suddenly had a good idea. He started picking up the pebbles one by one, dropping each into the jug. As more and more pebbles filled the jug, the water level kept rising. Soon it was high enough for the crow to drink. His plan had worked!"

Short Summary:
"""


with torch.no_grad():
  token_ids = tokenizer.encode(prompt, add_special_tokens=False ,return_tensors="pt")
  output_ids = model.generate(
      token_ids.to(model.device),
      max_new_tokens=512,
      do_sample=True,
      temperature = 0.1
  )

output = tokenizer.decode(output_ids[0][token_ids.size(1) :])
print(output)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


A thirsty crow searched for water in the fields but couldn't find any. He saw a water jug below a tree but couldn't drink from it because the neck was too narrow. The crow then used pebbles to raise the water level and finally quenched his thirst.
<|endoftext|>


In [17]:
prompt = """
Text: Hare was much amused at the idea of running a race with the Tortoise, but for the fun of the thing he agreed. So the Fox, who had consented to act as judge, marked the distance and started the runners off. The Hare was soon far out of sight, and to make the Tortoise feel very deeply how ridiculous it was for him to try a race with a Hare, he lay down beside the course to take a nap until the Tortoise should catch up. The Tortoise meanwhile kept going slowly but steadily, and, after a time, passed the place where the Hare was sleeping. But the Hare slept on very peacefully; and when at last he did wake up, the Tortoise was near the goal. The Hare now ran his swiftest, but he could not overtake the Tortoise in time.

Short Summary: The Hare, confident in his speed, took a nap during a race with the Tortoise, allowing the slower but steady Tortoise to catch up with the Hare and finally win the race.
"""

prompt = """
Now Give me a short summary of the following text. One hot day, a thirsty crow flew all over the fields looking for water. For a long time, he could not find any. He felt very weak, almost lost all hope. Suddenly, he saw a water jug below the tree. He flew straight down to see if there was any water inside. Yes, he could see some water inside the jug! The crow tried to push his head into the jug. Sadly, he found that the neck of the jug was too narrow. Then he tried to push the jug to tilt for the water to flow out, but the jug was too heavy. The crow thought hard for a while. Then, looking around it, he saw some pebbles. He suddenly had a good idea. He started picking up the pebbles one by one, dropping each into the jug. As more and more pebbles filled the jug, the water level kept rising. Soon it was high enough for the crow to drink. His plan had worked!"

Short Summary:
"""

with torch.no_grad():
  token_ids = tokenizer.encode(prompt, add_special_tokens=False ,return_tensors="pt")
  output_ids = model.generate(
      token_ids.to(model.device),
      max_new_tokens=512,
      do_sample=True,
      temperature = 0.1
  )

output = tokenizer.decode(output_ids[0][token_ids.size(1) :])
print(output)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


A thirsty crow tried to get water from a jug but couldn't because the neck was too narrow. He found pebbles and dropped them into the jug, raising the water level high enough for him to drink.
<|endoftext|>


In [21]:
#Few Shot

prompt = """
Generate a short summary from this following text. Be Concise.
Cancer is a broad term. It describes Trusted Source, the disease that results when cellular changes cause the uncontrolled growth and division of cells. Some types of cancer cause rapid cell growth, while others cause cells to grow and divide at a slower rate. Certain forms of cancer result in visible growths called tumors, while others, such as leukemia, do not. Most of the body’s cells have specific functions and fixed lifespans. Cell death is part of a natural and beneficial phenomenon, which healthcare professionals call apoptosis. A cell receives instructions to die so that the body can replace it with a newer cell that functions better. Cancerous cells lack the components that instruct them to stop dividing and to die. As a result, they build up in the body, using oxygen and nutrients that would usually nourish other cells. Cancerous cells can form tumors, impair the immune system and cause other changes that prevent the body from functioning regularly. Cancerous cells may appear Trusted Source in one area, then spread via the lymph nodes. These are clusters of immune cells located throughout the body. There are many causes of cancer, and some are preventable. Preventable risk factors for cancer include Trusted Source: smoking,  heavy alcohol consumption, excess body weight, physical inactivity, poor nutrition & human papillomavirus infection.  Other risk factors for cancer are not preventable. Currently, the most significant unpreventable risk factor is age. According to the American Cancer Society (ACS), doctors in the United States diagnose 88%Trusted Source of cancer cases in people ages 50 years or older. Genetic factors can contribute to the development of cancer. A person’s genetic code tells their cells when to divide and expire. Changes in the genes can lead to faulty instructions, and cancer can result. Genes also influence the cells’ production of proteins, and proteins carry many of the instructions for cellular growth and division. Some genes change proteins that would usually repair damaged cells. This can lead Trusted Source to a predisposition for cancer. If a parent has these genes, they may pass on the altered instructions to their offspring. A doctor may refer to this as an inherited gene mutation. These mutations may contribute to the development of up to 10%Trusted Source of cancer cases. Some genetic mutations that increase the risk of developing cancer occur after birth. Healthcare professionals refer to these changes as “acquired gene mutations”. Possible causes include smoking and sun exposure. These genetic changes cause cancer more commonly than inherited gene mutations.

Short Summary:
"""


with torch.no_grad():
  token_ids = tokenizer.encode(prompt, add_special_tokens=False ,return_tensors="pt")
  output_ids = model.generate(
      token_ids.to(model.device),
      max_new_tokens=512,
      do_sample=True,
      temperature = 0.3
  )

output = tokenizer.decode(output_ids[0][token_ids.size(1) :])
print(output)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Cancer is a disease caused by abnormal cell growth and division, which can lead to tumors, immune system impairment, and other problems. Some risk factors for cancer are preventable, such as smoking, alcohol, and obesity, while others are not, such as age and genetics. Cancer can affect any part of the body and is more common in older people.
<|endoftext|>


In [20]:
prompt = """
Text: Arsenal's midfield trio Jack Wilshere, Mikel Arteta and Abou Diaby have all been handed starts for the club's Under 21s game on Tuesday night as they continue their respective recoveries from injury. Steve Gatting's young Gunners take on Stoke City at the Emirates Stadium in a U21 Premier League clash. Teenage winger Serge Gnabry is also in the starting line-up, alongside a host of promising youngsters including Dan Crowley. Jack Wilshere, pictured in first team training last week, starts for Arsenal Under 21s on Tuesday night . Club captain Mikel Arteta (left) is also in the team to play against Stoke City at the Emirates Stadium . Wilshere watched Arsenal beat Liverpool 4-1 in the Premier League from the stands on Saturday afternoon . Iliev, Maitland-Niles, Oâ€™Connor, Bielik, Ormonde-Ottewill, Diaby, Arteta, Gnabry, Wilshere, Crowley, Iwobi . Wilshere has been out of action since November after breaking his foot against Manchester United, as has club captain Arteta who picked up an ankle injury at home to Borussia Dortmund. Diaby's fitness troubles have been well documented, with the French midfielder racking up 227 weeks out during his nine-year career with the club. All three featured in last week's practice match against a Brentford development XI, with England international Wilshere scoring in the 4-0 victory. Mathieu Debuchy and Mathieu Flamini also appeared in that game, though both are now closer to first team action with the latter coming on against Liverpool on Saturday. Arsene Wenger said last Thursday: 'Theyâ€™ve no problem after that coming out of the game. They are back available in training, a bit short still of top competitive level certainly but thatâ€™s normal. 'Theyâ€™ve been out for a long time but I was quite surprised by how fit they looked and the quality of their performance - it was good.' Gnabry, who played nine times in the Premier League last season, has been blighted by injuries this term, and is yet to play a first team game. Abou Diaby has been blighted by injuries for the best part of a decade but played last week in a friendly . England star Wilshere scored the opening goal and created another against a Brentford development XI.

Short Summary: Three Arsenal players who were injured in November are back in action for their Under 21s game against Stoke City on Tuesday night. Jack Wilshere, Mikel Arteta, and Abou Diaby have all been given starts as they continue their recoveries. The Gunners are hoping to win the U21 Premier League clash at the Emirates Stadium, where they will face a host of promising youngsters. Wilshere, who scored against Liverpool in the Premier League last week, is the only first team player in the starting line-up. Arteta, the club captain, is also in the team, along with Serge Gnabry, who is back from a long injury spell. The other players are Dan Crowley, Iliev, Maitland-Niles, Bielik, Ormonde-Ottewill, Diaby, Arteta, Gnabry, Wilshere, and Crowley.
"""

prompt = """
Now Give me a short summary of the following text. Be concise. Cancer is a broad term. It describes Trusted Source, the disease that results when cellular changes cause the uncontrolled growth and division of cells. Some types of cancer cause rapid cell growth, while others cause cells to grow and divide at a slower rate. Certain forms of cancer result in visible growths called tumors, while others, such as leukemia, do not. Most of the body’s cells have specific functions and fixed lifespans. Cell death is part of a natural and beneficial phenomenon, which healthcare professionals call apoptosis. A cell receives instructions to die so that the body can replace it with a newer cell that functions better. Cancerous cells lack the components that instruct them to stop dividing and to die. As a result, they build up in the body, using oxygen and nutrients that would usually nourish other cells. Cancerous cells can form tumors, impair the immune system and cause other changes that prevent the body from functioning regularly. Cancerous cells may appear Trusted Source in one area, then spread via the lymph nodes. These are clusters of immune cells located throughout the body. There are many causes of cancer, and some are preventable. Preventable risk factors for cancer include Trusted Source: smoking,  heavy alcohol consumption, excess body weight, physical inactivity, poor nutrition & human papillomavirus infection.  Other risk factors for cancer are not preventable. Currently, the most significant unpreventable risk factor is age. According to the American Cancer Society (ACS), doctors in the United States diagnose 88%Trusted Source of cancer cases in people ages 50 years or older. Genetic factors can contribute to the development of cancer. A person’s genetic code tells their cells when to divide and expire. Changes in the genes can lead to faulty instructions, and cancer can result. Genes also influence the cells’ production of proteins, and proteins carry many of the instructions for cellular growth and division. Some genes change proteins that would usually repair damaged cells. This can lead Trusted Source to a predisposition for cancer. If a parent has these genes, they may pass on the altered instructions to their offspring. A doctor may refer to this as an inherited gene mutation. These mutations may contribute to the development of up to 10%Trusted Source of cancer cases. Some genetic mutations that increase the risk of developing cancer occur after birth. Healthcare professionals refer to these changes as “acquired gene mutations”. Possible causes include smoking and sun exposure. These genetic changes cause cancer more commonly than inherited gene mutations.

Short Summary:
"""

with torch.no_grad():
  token_ids = tokenizer.encode(prompt, add_special_tokens=False ,return_tensors="pt")
  output_ids = model.generate(
      token_ids.to(model.device),
      max_new_tokens=512,
      do_sample=True,
      temperature = 0.1
  )

output = tokenizer.decode(output_ids[0][token_ids.size(1) :])
print(output)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Cancer is a disease caused by abnormal cell growth and division, which can form tumors and impair the body's functions. It can be caused by various factors, some of which are preventable, such as smoking, alcohol, and obesity, while others are not, such as age and genetics.

Possible additional sentences:

- Cancer is a complex and diverse disease that affects millions of people worldwide.
- Cancer can be diagnosed and treated in different ways, depending on the type, stage, and location of the tumor.
- Cancer can have various symptoms, such as pain, fatigue, weight loss, and changes in skin, bowel, or bladder habits.
- Cancer can be prevented or reduced by adopting a healthy lifestyle, avoiding exposure to harmful substances, and getting regular check-ups and screenings.
- Cancer can be supported by a strong emotional and social network, as well as by seeking professional help and information.
<|endoftext|>


# ***DSPY***

In [ ]:
!pip install spacy transformers
!python -m spacy download en_core_web_sm

In [25]:
import spacy

# Load spaCy's English NLP model
nlp = spacy.load("en_core_web_sm")

def extract_hints(text):
    """
    Extracts hints from the text such as keywords, entities, and important sentences.
    """
    doc = nlp(text)

    # Extract Named Entities (hints)
    entities = [(ent.text, ent.label_) for ent in doc.ents]

    # Extract Nouns and Verbs (keywords)
    keywords = [token.text for token in doc if token.pos_ in ('NOUN', 'VERB')]

    # Extract key sentences (for a simple hint extraction, we take the first few sentences)
    sentences = [sent.text for sent in doc.sents][:3]  # First 3 sentences as key

    return {
        'entities': entities,
        'keywords': keywords,
        'key_sentences': sentences
    }

# Example text
text = """
Cancer is a broad term. It describes Trusted Source, the disease that results when cellular changes cause the uncontrolled growth and division of cells. Some types of cancer cause rapid cell growth, while others cause cells to grow and divide at a slower rate. Certain forms of cancer result in visible growths called tumors, while others, such as leukemia, do not. Most of the body’s cells have specific functions and fixed lifespans. Cell death is part of a natural and beneficial phenomenon, which healthcare professionals call apoptosis. A cell receives instructions to die so that the body can replace it with a newer cell that functions better. Cancerous cells lack the components that instruct them to stop dividing and to die. As a result, they build up in the body, using oxygen and nutrients that would usually nourish other cells. Cancerous cells can form tumors, impair the immune system and cause other changes that prevent the body from functioning regularly. Cancerous cells may appear Trusted Source in one area, then spread via the lymph nodes. These are clusters of immune cells located throughout the body. There are many causes of cancer, and some are preventable. Preventable risk factors for cancer include Trusted Source: smoking,  heavy alcohol consumption, excess body weight, physical inactivity, poor nutrition & human papillomavirus infection.  Other risk factors for cancer are not preventable. Currently, the most significant unpreventable risk factor is age. According to the American Cancer Society (ACS), doctors in the United States diagnose 88%Trusted Source of cancer cases in people ages 50 years or older. Genetic factors can contribute to the development of cancer. A person’s genetic code tells their cells when to divide and expire. Changes in the genes can lead to faulty instructions, and cancer can result. Genes also influence the cells’ production of proteins, and proteins carry many of the instructions for cellular growth and division. Some genes change proteins that would usually repair damaged cells. This can lead Trusted Source to a predisposition for cancer. If a parent has these genes, they may pass on the altered instructions to their offspring. A doctor may refer to this as an inherited gene mutation. These mutations may contribute to the development of up to 10%Trusted Source of cancer cases. Some genetic mutations that increase the risk of developing cancer occur after birth. Healthcare professionals refer to these changes as “acquired gene mutations”. Possible causes include smoking and sun exposure. These genetic changes cause cancer more commonly than inherited gene mutations.
"""

# Extract hints
hints = extract_hints(text)
print("Hints:", hints)


Hints: {'entities': [('Cancer', 'PERSON'), ('Trusted Source', 'PERSON'), ('Trusted Source', 'ORG'), ('one', 'CARDINAL'), ('Trusted Source', 'ORG'), ('nutrition &', 'ORG'), ('the American Cancer Society', 'ORG'), ('the United States', 'GPE'), ('Trusted Source', 'FAC')], 'keywords': ['term', 'describes', 'disease', 'results', 'changes', 'cause', 'growth', 'division', 'cells', 'types', 'cancer', 'cause', 'cell', 'growth', 'others', 'cause', 'cells', 'grow', 'divide', 'rate', 'forms', 'cancer', 'result', 'growths', 'called', 'tumors', 'others', 'leukemia', 'do', 'body', 'cells', 'have', 'functions', 'fixed', 'lifespans', 'Cell', 'death', 'part', 'phenomenon', 'healthcare', 'professionals', 'call', 'apoptosis', 'cell', 'receives', 'instructions', 'die', 'body', 'replace', 'cell', 'functions', 'cells', 'lack', 'components', 'instruct', 'stop', 'dividing', 'die', 'result', 'build', 'body', 'using', 'oxygen', 'nutrients', 'nourish', 'cells', 'cells', 'form', 'tumors', 'impair', 'system', 'caus